In [16]:
!pip install pandas matplotlib seaborn --quiet


In [17]:
import pandas as pd

health_df = pd.read_csv('health_monitoring.csv')
safety_df = pd.read_csv('safety_monitoring.csv')
reminder_df = pd.read_csv('daily_reminder.csv')


In [18]:
class AgentBase:
    def __init__(self, name):
        self.name = name

    def run(self, data):
        raise NotImplementedError("Each agent must implement its own run method.")


In [19]:
class HealthMonitoringAgent(AgentBase):
    def run(self, health_df):
        def analyze(row):
            alerts = []
            if row["Heart Rate Below/Above Threshold (Yes/No)"] == "Yes":
                alerts.append("Heart Rate Alert")
            if row["Blood Pressure Below/Above Threshold (Yes/No)"] == "Yes":
                alerts.append("Blood Pressure Alert")
            if row["Glucose Levels Below/Above Threshold (Yes/No)"] == "Yes":
                alerts.append("Glucose Alert")
            if row["SpO₂ Below Threshold (Yes/No)"] == "Yes":
                alerts.append("Low SpO₂")
            return alerts if alerts else ["Normal"]

        health_df["Health Alerts"] = health_df.apply(analyze, axis=1)
        return health_df


In [20]:
class SafetyMonitoringAgent(AgentBase):
    def run(self, safety_df):
        def check(row):
            if row["Fall Detected (Yes/No)"] == "Yes" or row["Post-Fall Inactivity Duration (Seconds)"] > 60:
                return "Safety Alert"
            return "Safe"

        safety_df["Safety Status"] = safety_df.apply(check, axis=1)
        return safety_df


In [21]:
class ReminderAgent(AgentBase):
    def run(self, reminder_df):
        def evaluate(row):
            if row["Reminder Sent (Yes/No)"] == "Yes" and row["Acknowledged (Yes/No)"] == "No":
                return "Ignored"
            elif row["Reminder Sent (Yes/No)"] == "No":
                return "Not Sent"
            return "Acknowledged"

        reminder_df["Reminder Status"] = reminder_df.apply(evaluate, axis=1)
        return reminder_df


In [22]:
class CoordinatorAgent:
    def __init__(self, agents):
        self.agents = agents

    def coordinate(self, datasets):
        results = {}
        for agent in self.agents:
            print(f"Running {agent.name}...")
            results[agent.name] = agent.run(datasets[agent.name])
        return results


In [24]:
# Initialize agents
health_agent = HealthMonitoringAgent("health_df")
safety_agent = SafetyMonitoringAgent("safety_df")
reminder_agent = ReminderAgent("reminder_df")

# Pass to coordinator
coordinator = CoordinatorAgent([health_agent, safety_agent, reminder_agent])

# Prepare datasets
datasets = {
    "health_df": health_df,
    "safety_df": safety_df,
    "reminder_df": reminder_df
}

# Run coordination
results = coordinator.coordinate(datasets)

# You can now access:
# results["health_df"], results["safety_df"], results["reminder_df"]


Running health_df...
Running safety_df...
Running reminder_df...
